In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split
import random
# Building a english to french translator

C:\Python\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# data_path = 'fra.txt'
data_path = 'joey.npy'
if '.txt' in data_path:
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for idx,line in enumerate(lines):
        lines[idx]="\t".join(line.split('\t')[0:-1])
if '.npy' in data_path:
    lines = str(np.load(data_path)).split('\n')

In [3]:
input_lines = list()
target_lines = list()
# for line in random.choices(lines,k=10000):
for line in lines[0:500]:
    if len(line):
        inp,target = line.split('\t')
        input_lines.append(inp)
        target_lines.append('START_ '+ target + ' _END')
    
all_input_words=set()
for inp in input_lines:
    for word in inp.split():
        if word not in all_input_words:
            all_input_words.add(word)
    
all_target_words=set()
for tar in target_lines:
    for word in tar.split():
        if word not in all_target_words:
            all_target_words.add(word)

In [4]:
input_words = sorted(list(all_input_words))
target_words = sorted(list(all_target_words))

num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)

max_encoder_seq_length = max([len(l.split(' ')) for l in input_lines])
max_decoder_seq_length = max([len(l.split(' ')) for l in target_lines])

print('Number of samples:', len(input_lines))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

Number of samples: 500
Number of unique input tokens: 1511
Number of unique output tokens: 1550
Max sequence length for inputs: 54
Max sequence length for outputs: 111


In [5]:
encoder_input_data = np.zeros(
    (len(input_lines), max_encoder_seq_length),#, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_lines), max_decoder_seq_length),#, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_lines), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

* encoder_input_data ::: (Max sequence length for inputs, Number of unique input tokens)
* decoder_input_data ::: (Max sequence length for outputs, Number of unique output tokens)

In [6]:
for i, (input_text, target_text) in enumerate(zip(input_lines, target_lines)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [34]:
decoder_input_data[0]

array([  22.,  910.,  605.,  191.,  130.,  608.,  673., 1537.,  608.,
        673., 1023.,   23.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.], dtype=float32)

In [36]:
[idx for idx,i in enumerate(decoder_target_data[0][0]) if i==1]

[910]

In [12]:
len(decoder_target_data[0][1])

1550

In [7]:
input_lines[200].split()

['you', 'hand-wrote', 'it?']

In [8]:
target_lines[200]

"START_ yeah, and don't worry. i didn't try to sound smart at all! see ya later! _END"

In [9]:
encoder_input_data[200]

array([1493.,  547.,  674.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
      dtype=float32)

In [10]:
decoder_input_data[200]

array([  22., 1522.,   68.,  321., 1504.,  605.,  301., 1342., 1319.,
       1181., 1149.,   99.,   51., 1107., 1515.,  692.,   23.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.], dtype=float32)

# Building Model

`Embedding(
            embeddings.shape[0],
            embeddings.shape[1],
            input_length=shape["max_length"],
            trainable=False,
            weights=[embeddings],
            mask_zero=True,
        )`

In [18]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
import keras

In [19]:
embedding_size = 50
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [20]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)

decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
# model.summary()

In [21]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=5,
          validation_split=0.05)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 475 samples, validate on 25 samples
Epoch 1/5
475/475 [==============================] - 5s 11ms/step - loss: 0.7064 - acc: 0.0050 - val_loss: 0.8717 - val_acc: 0.0090
Epoch 2/5
475/475 [==============================] - 4s 9ms/step - loss: 0.7014 - acc: 0.0090 - val_loss: 0.8505 - val_acc: 0.0094
Epoch 3/5
475/475 [==============================] - 4s 8ms/step - loss: 0.6719 - acc: 0.0090 - val_loss: 0.8157 - val_acc: 0.0090
Epoch 4/5
475/475 [==============================] - 4s 7ms/step - loss: 0.6446 - acc: 0.0090 - val_loss: 0.7961 - val_acc: 0.0090
Epoch 5/5
475/475 [==============================] - 4s 8ms/step - loss: 0.6267 - acc: 0.0090 - val_loss: 0.7821 - val_acc: 0.0090


In [15]:
# modelLoad = keras.models.load_model('word50.h5')

In [28]:
model.weights

[<tf.Variable 'embedding_2/embeddings:0' shape=(1511, 50) dtype=float32_ref>,
 <tf.Variable 'embedding_3/embeddings:0' shape=(1550, 50) dtype=float32_ref>,
 <tf.Variable 'lstm_2/kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_2/recurrent_kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_2/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'lstm_3/kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_3/recurrent_kernel:0' shape=(50, 200) dtype=float32_ref>,
 <tf.Variable 'lstm_3/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(50, 1550) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(1550,) dtype=float32_ref>]

In [29]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())

reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
states_value = encoder_model.predict(input_seq)

In [30]:
encoder_model = Model(encoder_inputs, encoder_states)
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [21]:

for seq_index in [1,2,3,4]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_lines[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['hey, you know, before you said that nothing could happen between us? what changed?']
Decoded sentence:  i _END
-
Input sentence: ['rach, you there?']
Decoded sentence:  i _END
-
Input sentence: ["oh, ju-ju-just stay calm. just be calm. for all he knows we're just hanging out together. right? just be nonchalant.  that's not nonchalant!"]
Decoded sentence:  i _END
-
Input sentence: ['yeah, sure...']
Decoded sentence:  i _END
